In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from datetime import timedelta
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse

In [ ]:
from sqlalchemy.sql import label
from sqlalchemy import *

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine,reflect = True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
recent_date = session.query(Measurement).order_by(Measurement.date.desc()).first()
print(f"The most recent date in the data set is: {recent_date.date}")

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
most_recent_date = parse(recent_date.date)
# Calculate the date one year from the last date in data set.
start_date = most_recent_date  + relativedelta(months=-12) - timedelta(days=1)

# Perform a query to retrieve the data and precipitation scores
query_result = session.query(Measurement.date,Measurement.prcp).\
                filter(Measurement.date.between(start_date,most_recent_date)).all()


# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(query_result, columns=['date', 'prcp'])
df.set_index('date', inplace=True,)

# Sort the dataframe by date
df = df.sort_values(["date"])
sorted_df = df.reset_index(drop=False)
sorted_df.tail()


In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
x_axis = sorted_df.date
y_axis = sorted_df.prcp

fig, ax = plt.subplots(figsize=(20, 8))
ax.bar(x_axis,y_axis, label = 'Precipitation')
ax.set_xlabel("Date")
ax.set_ylabel("Inches")
ax.set_title("Last 12 months of Precipitation Data")
plt.legend(loc='best')

ax.xaxis.set_major_locator(mdates.DayLocator(interval=30))
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

plt.tight_layout()
plt.savefig("Images/Precipitation.png")
plt.show()


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
sorted_df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
station_count = session.query(Station).count()
print(f"The total number of stations in the dataset is: {station_count}")

In [ ]:

# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
active_station = session.query(Measurement.station, func.count().label("measurement_count"))\
                                                        .group_by(Measurement.station)\
                                                        .order_by(desc(func.count())).first()

print(f'The most active station is: {active_station.station}')

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
most_active_station = active_station.station
print(f'The most active station is: {most_active_station}')
analyse_station = session.query(Measurement, \
                               func.min(Measurement.tobs).label("mini_temp"),\
                                func.max(Measurement.tobs).label("max_temp"),\
                                func.avg(Measurement.tobs).label("average_temp")).filter(Measurement.station == most_active_station).all()
for result in analyse_station:
    print(f"The lowest temperature recorded: {result.mini_temp}")
    print(f"The highest temperature recorded: {result.max_temp}")
    print(f"The average temperature recorded: {result.average_temp}")


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
active_station_tob = session.query(Measurement.tobs).\
                filter(Measurement.date.between(start_date,most_recent_date)).\
                filter(Measurement.station == most_active_station).all()

print(active_station_tob)

In [ ]:
hist_df = pd.DataFrame(active_station_tob, columns=['tobs'])
#hist_df.head(10)

temp_hist_plot = hist_df.hist(column = 'tobs',bins = 12, grid=True, xlabelsize = 12, ylabelsize = 12,\
        figsize=(12, 6),color = 'red',alpha = 0.55,legend = True)

plt.title(f'Last 12 months of Temperature Observation Data:{most_active_station}',fontdict={'fontsize':18})
plt.xlabel('Temperature',fontdict={'fontsize':15})
plt.ylabel('Frequency',fontdict={'fontsize':15})
plt.grid(axis = 'y',color='gray', linestyle='dashed')
plt.grid(axis = 'x',color='gray', linestyle='dashed')

plt.tight_layout()
plt.savefig("Images/station_histogram.png")
plt.show()


# Close session

In [ ]:
# Close Session
session.close()